In [12]:
!python Feature_Extract.py aes128_table_ecb key_in

!python Feature_Extract.py AES_PPRM1 Kin AES_PPRM1
!python Feature_Extract.py AES_PPRM3 Kin AES_PPRM3
!python Feature_Extract.py AES_TBL Kin AES_TBL
!python Feature_Extract.py RSA Kin RSA
!python Feature_Extract.py SABER pol_64bit_in SABER
!python Feature_Extract.py aes128_table_ecb key_in train

[INFO] Features written to ../out/features.csv
[INFO] Edges written to ../out/edges.csv
[INFO] Features written to ../test/AES_PPRM1_features.csv
[INFO] Edges written to ../test/AES_PPRM1_edges.csv
[INFO] Features written to ../test/AES_PPRM3_features.csv
[INFO] Edges written to ../test/AES_PPRM3_edges.csv
[INFO] Features written to ../test/AES_TBL_features.csv
[INFO] Edges written to ../test/AES_TBL_edges.csv
[INFO] Features written to ../test/RSA_features.csv
[INFO] Edges written to ../test/RSA_edges.csv
[INFO] Features written to ../test/SABER_features.csv
[INFO] Edges written to ../test/SABER_edges.csv
[INFO] Features written to ../test/train_features.csv
[INFO] Edges written to ../test/train_edges.csv


In [25]:
!python SCAR_GNN.py

/Users/ruixiang/PycharmProjects/SCAR/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
num_features: 7, num_classes: 2
feature_names: ['Degree', 'Hamming distance', 'Paths', 'and', 'mux', 'or', 'xor']
Train label distribution:
label
0    138
1     43
Name: count, dtype: int64

Test label distribution:
label
0    35
1    11
Name: count, dtype: int64
Epoch 1/32
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - acc: 0.6716 - loss: 4.4219 - precision: 0.1687 - recall: 0.1158 - val_acc: 0.6316 - val_loss: 3.4865 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/32
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.6914 - loss: 3.9481 - precision: 0.2260 - recall: 0.1531 - val_acc: 0.6316 - val_loss: 3.7662 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/32
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2m

In [26]:
!python test.py

/Users/ruixiang/PycharmProjects/SCAR/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Found 6 dataset(s): ['AES_PPRM1', 'AES_PPRM3', 'AES_TBL', 'RSA', 'SABER', 'train']
Testing AES_PPRM1: read ../test/AES_PPRM1_features.csv and ../test/AES_PPRM1_edges.csv
Tensor("GatherV2:0", shape=(1, 32), dtype=float32)
Tensor("gnn_model_1/GatherV2:0", shape=(1, 32), dtype=float32)
Tensor("gnn_model_1/GatherV2:0", shape=(1, 32), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step
Tensor("gnn_model_1/GatherV2:0", shape=(None, 32), dtype=float32)
Testing AES_PPRM3: read ../test/AES_PPRM3_features.csv and ../test/AES_PPRM3_edges.csv
Tensor("GatherV2:0", shape=(1, 32), dtype=float32)
Tensor("gnn_model_1/GatherV2:0", shape=(1, 32), dtype=float32)
Tensor("gnn_model_1/GatherV2:0", shape=(1, 32), dtype=float32